# Introduction

This notebook answers some of the most common "How To.." questions that pop-up during data cleaning.  
I've also cleaned this data in R. You may check out the R version [here on RPubs](https://rpubs.com/Shefali/cleaning-audible-dataset).  

I hope both versions of this work helps beginners in understanding corresponding functions in Python and R.

The dataset used in this notebook can be found [here on Kaggle](https://www.kaggle.com/datasets/snehangsude/audible-dataset/data).

<a id="0"></a> <br>  
# Table of Contents 
1. [How to check total count of each category?](#1)  
2. [How to check number of duplicate rows?](#2)  
3. [How to remove a string pattern?](#3)  
4. [How to add space between 2 or more word blocks? (here first, middle, last name)](#4)  
5. [How to split a value into multiple columns and assign column name with prefix/suffix?](#5)  
6. [ How to find total number of NA in each column?](#6)  
7. [How to extract digits before a specifc word?](#7)  
8. [How to convert NaN values to 0?](#8)
9. [How to change the datatype of multiple columns?](#9)  
10. [How to check for inconsistencies in date column stored as a string?](#10)  
11. [How to convert date stored as string to 'date' type?](#11)  
12. [How to convert text to one uniform case?](#12)  
13. [How to split a text column with a specific word acting as separator?](#13)  
14. [How to extract decimal numbers from a text?](#14)

In [2]:
#load libraries
import pandas as pd
import numpy as np
import random

The code below increases the number of rows displayed in jupyter notebook when printing a dataframe.

In [3]:
#increase number of rows displayed by default in jupyter notebook
pd.options.display.max_rows = 50000

In [4]:
#read data
audible_data = pd.read_csv("data/audible_dataset/audible_uncleaned.csv")

In [7]:
#preview of the data
audible_data.head(4)

,name,author,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,Writtenby:GeronimoStilton,Narratedby:BillLobely,2 hrs and 20 mins,04-08-08,English,5 out of 5 stars34 ratings,468.00
1,The Burning Maze,Writtenby:RickRiordan,Narratedby:RobbieDaymond,13 hrs and 8 mins,01-05-18,English,4.5 out of 5 stars41 ratings,820.00
2,The Deep End,Writtenby:JeffKinney,Narratedby:DanRussell,2 hrs and 3 mins,06-11-20,English,4.5 out of 5 stars38 ratings,410.00
3,Daughter of the Deep,Writtenby:RickRiordan,Narratedby:SoneelaNankani,11 hrs and 16 mins,05-10-21,English,4.5 out of 5 stars12 ratings,615.00


From the sample data above and information below, we can list out some inconsistencies in the dataframe.  
1. **author-** "Writtenby:" can be removed; multiple author names can be placed in different columns.  
2. **narrator-** "Narratedby:" can be removed.  
3. **time-** is of type `str`; can be converted to numeric or datetime type. We'll see what should be done here.  
4. **releasedate-** is of type `str`; should be of type `date`.  
5. **language-** all values should be in uniform case; first letter capital, rest smallcase.  
6. **stars-** can be split into i) number of stars and ii) total number of ratings columns.  
7. **price-** is of type `str`; should be float.

In [6]:
#information about the data
audible_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         87489 non-null  object
 1   author       87489 non-null  object
 2   narrator     87489 non-null  object
 3   time         87489 non-null  object
 4   releasedate  87489 non-null  object
 5   language     87489 non-null  object
 6   stars        87489 non-null  object
 7   price        87489 non-null  object
dtypes: object(8)
memory usage: 5.3+ MB


In [8]:
#stats
audible_data.describe()

,name,author,narrator,time,releasedate,language,stars,price
count,87489,87489,87489,87489,87489,87489,87489,87489
unique,82767,48374,29717,2284,5058,36,665,1011
top,The Art of War,"Writtenby:矢島雅弘,石橋遊",Narratedby:anonymous,2 mins,16-05-18,English,Not rated yet,586.00
freq,20,874,1034,372,773,61884,72417,5533


<a id = "1"></a>
## 1. How to check total count of each category?

In [9]:
#checking number of occurances of books
book_counts = audible_data['name'].value_counts()
#list out books which have 10 or more occurances
book_counts[book_counts >= 10]

The Art of War                 20
Sterling Biographies           19
The Odyssey                    16
Sterling Point Books           16
Hamlet                         15
The Prophet                    14
Pride and Prejudice            14
A Christmas Carol              14
The Iliad                      13
As a Man Thinketh              13
The Science of Getting Rich    13
The Picture of Dorian Gray     12
Abraham Lincoln                12
Meditations                    11
The Richest Man in Babylon     11
The Raven                      11
The Prince                     11
Unstoppable                    10
Name: name, dtype: int64

<a id = "2"></a>
## 2. How to check number of duplicate rows?

In [10]:
#check for duplicate rows
audible_data.duplicated().sum()

0

<a id = "3"></a>
## 3. How to remove a string pattern?

Changes mades to the **author** column:  
1. Removed "Writtenby:".  
2. Add space between first, middle(if any) and last names of the authors.  
3. Since some books have multiple authors, this column is split into multiple columns with 1 author name in each.  

The 3rd step is optional.  
I've added it here just to showcase how to split a text column into multiple columns, and assign column names automatically using prefixes like author1, author2 etc. 

In [13]:
#Author column
#remove the phrase "Writtenby:"
audible_data['author'] = audible_data['author'].str.replace(pat = "Writtenby:", repl = "")

In [14]:
#after removing "Written By", author column looks like:
audible_data.author.sample(5)

26665    AdrianLangenscheid
83566        HeatherFawcett
23713           PauldeKruif
20365          H.A.L.Fisher
60248      ArthurConanDoyle
Name: author, dtype: object

<a id = "4"></a>
## 4. How to add space between 2 or more word blocks? (here first, middle, last name)

In [15]:
#Add space between the first, middle and last names of Authors.
#e.g. JaneAustin becomes Jane Austin
audible_data['author'] = audible_data['author'].str.replace(pat = r"(\w)([A-Z])", repl = r"\1 \2", regex = True)

In [16]:
audible_data.author.sample(5)

47998                       Roald Dahl
28575    Travis Nicholson,Oliver Walsh
6387                          Sam Kean
69583                  Charles Freeman
22657                   Sarah Kendzior
Name: author, dtype: object

<a id = "5"></a>
## 5. How to split a value into multiple columns and assign column name with prefix/suffix?

In [17]:
audible_data2 = pd.concat( [audible_data['name'], 
                            #splits author into author1, author2 and author3.
                            audible_data['author'].str.split(',', expand = True).add_prefix('author'),
                            audible_data.loc[:,['narrator', 'time', 'releasedate', 'language', 'stars', 'price']]], 
                            axis = 1)

<a id = "6"></a>
## 6. How to find total number of NA in each column?

Here, I wanted to see if it's at all useful to split the author column or not.  
As we can see from the NA value counts below, the newly created columns ***author1, author2, author3*** are mostly vacant.  
Majority of the books have only 1 author and about 14,000 books have 2 authors.  
(Total number of unique books in the data- 82767	)

In [18]:
#number of missing values in author columns
audible_data2.loc[:,['author0','author1','author2','author3']].isnull().sum()

author0        0
author1    73762
author2    85135
author3    86713
dtype: int64

#### Changes made to narrator column:  
1. Removed "Narratedby:".  
2. Add space between first, middle and last names of the narrator.  
3. Here, I haven't split this column for multiple narrators.  


In [19]:
#remove "Narratedby:" from Narrator column
audible_data2['narrator'] = audible_data2['narrator'].str.replace(pat = "Narratedby:", repl = "")

In [20]:
#add space between first and last name of the Narrator
#e.g. JaneAustin becomes Jane Austin
audible_data2['narrator'] = audible_data2['narrator'].str.replace(pat = r"(\w)([A-Z])", repl = r"\1 \2", regex = True)

In [22]:
audible_data2.tail(5)

,name,author0,author1,author2,author3,narrator,time,releasedate,language,stars,price
87484,Last Days of the Bus Club,Chris Stewart,None,None,None,Chris Stewart,7 hrs and 34 mins,09-03-17,English,Not rated yet,596.00
87485,The Alps,Stephen O'Shea,None,None,None,Robert Fass,10 hrs and 7 mins,21-02-17,English,Not rated yet,820.00
87486,The Innocents Abroad,Mark Twain,None,None,None,Flo Gibson,19 hrs and 4 mins,30-12-16,English,Not rated yet,938.00
87487,A Sentimental Journey,Laurence Sterne,None,None,None,Anton Lesser,4 hrs and 8 mins,23-02-11,English,Not rated yet,680.00
87488,Havana,Mark Kurlansky,None,None,None,Fleet Cooper,6 hrs and 1 min,07-03-17,English,Not rated yet,569.00


### Time column

1. This column contains duration of audiobooks in text format.  
2. Some rows contain 'xx hrs and yy mins' while others contain 'y mins', 'less than 1 minute' etc.  
        In order to see all unique formats present, a copy of the time column has been created.

In [92]:
#make a copy of time column to understand all kind of formats in which data is present
time_column = audible_data2['time']

All digits have been removed to see what all phrases are present.

In [24]:
#replace all numbers with blanks
time_column = time_column.str.replace(pat = r'[0-9]', repl = '', regex = True)
#keep only unique patterns
time_column.drop_duplicates()

0           hrs and  mins
4                     hrs
12           hrs and  min
29           hr and  mins
53                   mins
227                    hr
255           hr and  min
1203                  min
1401    Less than  minute
Name: time, dtype: object

- **`Assumption:`** All rows with ***Less than xx minutes*** will be replaced with ***xx minutes***.  
- Below, I've filtered out all row indices which have this 'less than..' values to see all distinct durations.  
- Since only 1 unique value present, ***less than 1 minute***, it can be simply replaced with '1'.  
- If other values were present like ***less than 5 minutes, less than 10 mins*** etc., then we would have used regular expressions to extract and store the digits.

In [25]:
#find rows with "less than x minute" values in audiobook duration
less_than_duration = audible_data2[audible_data2.time.str.contains(pat = 'less than', case = False)].index
#check all unique values with 'less than' pattern
audible_data2.time[less_than_duration].drop_duplicates()

1401    Less than 1 minute
Name: time, dtype: object

The objective is to:  
1. Create 2 columns **`hour_component`** and **`min_component`** to store the hour and minute duration of the audiobook.  
2. Examples:  
     - ***'17 hrs and 5 mins'*** will split into 17 in hour column and 5 in minute column.  
     - ***'24 mins'*** will be stored as 0 in hour column and 24 in minute column.  
     - ***'less than 1 minute'*** will be stored as 0 in hour and 1 in minute column.
        

In [26]:
#create 2 columns- 1 for hour component and the other for minute component.
audible_data2['hour_component'] = 0
audible_data2['min_component'] = 0

<a id = "7"></a>
## 7. How to extract digits before a specifc word?

Regular Expression used:  
1. **"^-"** indicates start matching the pattern from beginning of the string.  
2. **"(\d+)"-** is the capture group. Means look for one or more digits.  
3. **+** means one or more occurances of digits.  
4. **"(\d+) hr"** captures digits before the word hr or hrs.  
5. **"(\d+) min"** captures digits before min/mins/minutes words.

In [27]:
#Extract the hour component of audiobook duration into hour column and minute into min_component column
audible_data2['hour_component'] = audible_data2.time.str.extract(pat = r'^(\d+) hr')
audible_data2['min_component'] = audible_data2.time.str.extract(pat = r'(\d+) min')

In [28]:
#preview of extracted time
audible_data2.loc[:,['time', 'hour_component', 'min_component']].sample(6)

,time,hour_component,min_component
976,5 hrs and 23 mins,5,23
44557,26 hrs and 28 mins,26,28
83223,4 hrs and 28 mins,4,28
23528,3 hrs and 39 mins,3,39
67605,7 hrs and 52 mins,7,52
69249,39 hrs and 54 mins,39,54


<a id = "8"></a>
## 8. How to convert NaN values to 0?

Now, after digits extraction, values like ***'24 mins'*** will keep 24 in minute column and NaN in hour column.  
Similarly, a value like ***'11 hrs'*** will keep NaN in minutes column.  
Below, these NaN values have been replaced with 0.

In [29]:
#convert all NaN values in hour and minute component column to 0
audible_data2['hour_component'] = audible_data2['hour_component'].fillna(0)
audible_data2['min_component'] = audible_data2['min_component'].fillna(0)

In [31]:
#sample preview-- some random indices to show how values are tranformed
audible_data2.loc[[328,5532,1583], ['time', 'hour_component', 'min_component']]

,time,hour_component,min_component
328,11 mins,0,11
5532,12 mins,0,12
1583,5 mins,0,5


<a id = "9"></a>
## 9. How to change the datatype of multiple columns?

Now, we can change the datatype of hour and minutes from strings to integers. 

In [32]:
#convert the hour and min column to int type
audible_data2 = audible_data2.astype({'hour_component':'int','min_component':'int'})

In [33]:
audible_data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            87489 non-null  object
 1   author0         87489 non-null  object
 2   author1         13727 non-null  object
 3   author2         2354 non-null   object
 4   author3         776 non-null    object
 5   narrator        87489 non-null  object
 6   time            87489 non-null  object
 7   releasedate     87489 non-null  object
 8   language        87489 non-null  object
 9   stars           87489 non-null  object
 10  price           87489 non-null  object
 11  hour_component  87489 non-null  int32 
 12  min_component   87489 non-null  int32 
dtypes: int32(2), object(11)
memory usage: 8.0+ MB


### Date column

Dates are written in dd-mm-yy format.  
They are in character format and will be converted to date type.  
But before that, we need to make sure that each of the components are consistent. That is:  
    - **dd** doesn't exceed 31.  
    - **mm** doesn't exceed 12.  
    - In the [datacard](https://www.kaggle.com/datasets/snehangsude/audible-dataset/data) on Audible dataset page, its mentioned that some books are yet to be released, which means we can have year value greater than 2023.

In [35]:
audible_data2.releasedate.sample(5)

22624    01-05-12
65185    07-07-21
47740    14-09-21
8098     02-09-21
10972    11-10-11
Name: releasedate, dtype: object

<a id = "10"></a>
## 10. How to check for inconsistencies in date column stored as a string?

Regular expressions used:  
1. **"^(\d+)"-** check for the digit from beginning of string (^) and before hypher -.  
2. **"-(\d+)-"** check for the digits between the two hyphens.  
3. **"-(\d+)\$"-** capture the digits after hyphen sign and end of the string. 

In [40]:
#day component- dd
#check if any value in day part > 31.
any(audible_data2.releasedate.str.extract(pat = r'^(\d+)-').drop_duplicates().astype(int) > 31)

False

In [41]:
#check for inconsistencies in month of release date
#Check to see whether middle values (months) contain any number > 12
#all unique months. 
any((audible_data2.releasedate.str.extract(pat = r'-(\d+)-').drop_duplicates()).astype(int) > 12)

False

In [42]:
#check inconsistencies in year value of release date
#future release date is also present, hence values 25 for year 2025, 24 for year 2024...
audible_data2.releasedate.str.extract(pat = r'-(\d+)$').drop_duplicates()

,0
0,08
1,18
2,20
3,21
4,10
6,14
7,17
9,19
11,11
16,04


<a id = "11"></a>
## 11. How to convert date stored as string to 'date' type? 

Note: Check the separator used between day, month and year.  

- If date is like- 02/10/2023, use the format- **%d/%m/%Y**.  
- If date is like- 02-10-2023, use the format- **%d-%m-%Y**.  
- If year has 2 digits as in 02-10-23, use format- **%d-%m-%y**.  
- If year has 4 digits as in 02-10-2023, use Y in capital- **%d-%m-%Y**.

In [43]:
#convert release-date from string to date type
audible_data2.releasedate = pd.to_datetime(audible_data2.releasedate, format = '%d-%m-%y')

In [44]:
audible_data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   name            87489 non-null  object        
 1   author0         87489 non-null  object        
 2   author1         13727 non-null  object        
 3   author2         2354 non-null   object        
 4   author3         776 non-null    object        
 5   narrator        87489 non-null  object        
 6   time            87489 non-null  object        
 7   releasedate     87489 non-null  datetime64[ns]
 8   language        87489 non-null  object        
 9   stars           87489 non-null  object        
 10  price           87489 non-null  object        
 11  hour_component  87489 non-null  int32         
 12  min_component   87489 non-null  int32         
dtypes: datetime64[ns](1), int32(2), object(10)
memory usage: 8.0+ MB


<a id = "12"></a>
## 12. How to convert text to one uniform case? 

In [45]:
#language column
print(audible_data2.language.drop_duplicates().tolist())

['English', 'Hindi', 'spanish', 'german', 'french', 'catalan', 'swedish', 'italian', 'danish', 'finnish', 'dutch', 'hebrew', 'russian', 'polish', 'galician', 'afrikaans', 'icelandic', 'romanian', 'japanese', 'tamil', 'portuguese', 'urdu', 'hungarian', 'czech', 'bulgarian', 'mandarin_chinese', 'basque', 'korean', 'arabic', 'greek', 'turkish', 'ukrainian', 'slovene', 'norwegian', 'telugu', 'lithuanian']


In [46]:
#make language names uniform by capitalizing the first letter.
audible_data2.language = audible_data2.language.str.title()

### Stars column

<a id = "13"></a>
## 13. How to split a text column with a specific word acting as separator?

The objective is to create 2 columns out of the stars column:  
- **`stars_out_of_5`-** will contain numeric values like 2, 4.5 etc.  
- **`total_ratings`-** total number of reviews received on the particular audiobook.   

- Example: ***4.5 out of 5 stars7 ratings*** 
<br>
- This column will be split after the word 'stars'.  
- Then the first number before **'out'** will be extracted and converted to float type.  
- In the `total_ratings` column, **"ratings"** will be removed and values will be converted to float type.  

In [47]:
#unique stars
audible_data2.stars.sample(5)

19265                  Not rated yet
71010                  Not rated yet
53743                  Not rated yet
44762    4.5 out of 5 stars7 ratings
57766                  Not rated yet
Name: stars, dtype: object

In [48]:
#split stars column into 2
audible_data2[['stars_out_of_5', 'total_ratings']] = audible_data2.stars.str.split('stars', expand = True)

In [51]:
#glimpse of the separated values
audible_data2.loc[:,['stars','stars_out_of_5', 'total_ratings']].head(4)

,stars,stars_out_of_5,total_ratings
0,5 out of 5 stars34 ratings,5 out of 5,34 ratings
1,4.5 out of 5 stars41 ratings,4.5 out of 5,41 ratings
2,4.5 out of 5 stars38 ratings,4.5 out of 5,38 ratings
3,4.5 out of 5 stars12 ratings,4.5 out of 5,12 ratings


The value ***"Not rated yet"*** is being replaced with NA. 

In [ ]:
#replace 'Not rated yet' rows with NA

#rows with not yet rated value
not_rated_rows = audible_data2[audible_data2.stars_out_of_5.str.contains(pat = 'Not rated yet', case = False)].index
#replace 'Not yet rated' and 'None' with NA
audible_data2.loc[not_rated_rows,['stars_out_of_5', 'total_ratings']] = np.nan

#### Steps taken below:  

1. Remove ***'out of 5'*** from `stars_out_of_5` column to retain only the digit part.  
2. Remove ***'ratings'*** from `total_ratings` column to retain only the numeric part.  
3. Remove commas from values like ***1,500*** in `total_ratings` column. (Bcz this column has to be converted to numeric.)

In [54]:
#replace 'out of 5' with ''
audible_data2.stars_out_of_5 = audible_data2.stars_out_of_5.str.replace(pat = ' out of 5', repl = '')
#replace 'ratings' with ''
audible_data2.total_ratings = audible_data2.total_ratings.str.replace(pat = r' ratings| rating', repl = '', regex = True)
#remove ',' from ratings. e.g. 1,200 becomes 1200
audible_data2.total_ratings = audible_data2.total_ratings.str.replace(pat = ',', repl = '')

**NOTE:** **`total_ratings`** has been converted to float and not int because it contains NaN values (for rows with "not rated yet" books). Converting this column to int throws error.

In [57]:
#convert both columns to float type
audible_data2 = audible_data2.astype({'stars_out_of_5':'float', 'total_ratings':'float'})

### Price column  

#### Steps taken:

1. Replace 'Free' with 0.  
2. Remove commas from prices. Currently, this column is of type str; it has to be converted to numeric.  
3. Convert to float.

In [ ]:
#convert price to float type

#replace 'free' with 0
audible_data2.price = audible_data2.price.str.replace(pat = 'free', repl = '0', case = False)
#remove ',' from the values
audible_data2.price = audible_data2.price.str.replace(pat = ',', repl = '')
#convert to float
audible_data2.price = audible_data2.price.astype(float)

I've renamed some columns just to maintain consistency.

In [59]:
#rename columns author0 to author1, author1 to author2 etc.
audible_data2.rename(columns = {'author0':'author_1', 'author1':'author_2', 'author2':'author_3', 'author3':'author_4',
                               'releasedate': 'release_date'}, inplace = True)

And finally, I've created a subset of the dataframe with all relevant columns.

In [61]:
#final dataframe with relevant columns
cleaned_df = audible_data2.loc[:,['name', 'author_1', 'author_2', 'author_3', 'author_4', 'narrator', 'release_date',
                                 'hour_component', 'min_component', 'language', 'stars_out_of_5', 'total_ratings', 'price']]

In [62]:
cleaned_df.head(3)

,name,author_1,author_2,author_3,author_4,narrator,release_date,hour_component,min_component,language,stars_out_of_5,total_ratings,price
0,Geronimo Stilton #11 & #12,Geronimo Stilton,None,None,None,Bill Lobely,2008-08-04,2,20,English,5.0,34.0,468.00
1,The Burning Maze,Rick Riordan,None,None,None,Robbie Daymond,2018-05-01,13,8,English,4.5,41.0,820.00
2,The Deep End,Jeff Kinney,None,None,None,Dan Russell,2020-11-06,2,3,English,4.5,38.0,410.00


<a id = "14"></a>
## 14. How to extract decimal numbers from a text?

I used this step in initial phase of cleaning this dataframe, but then dropped it from the final draft.  
But since this notebook contains "How To.." steps, I've mentioned this step below just for the sake of future reference.

1. **`stars_out_of_5` column-** example- ***4.5 out of 5***.
     - **What I've done above**- replace 'out of 5' with '', then convert 4.5 from string to float.  
     - **Another way**- extract the floating-point number from this text, i.e. 4.5 and then convert it to float.
     

In [63]:
#example
ratings = pd.Series(['4.5 out of 5', '3 out of 5', '.5 out of 5'])

Regular expression used:  

1. **\[0-9\]\*-** means 0 or more occurance of a digit.  
2. **"\.?"-** means presence of a decimal point is optional.  
3. So, **"\[0-9\]\*\.?"** means 0 or more occurances of digits before decimal point.  
This pattern helps to capture values like **.5** where there are no digits before decimal.  
4. **"\.?\[0-9\]+"** ensures that there must be one or more digits after the decimal point.

In [91]:
#extract the rating received out of 5
ratings.str.extract(pat = r'([0-9]*\.?[0-9]+)')

,0
0,4.5
1,3
2,.5


So, this is it!  
Ofc, there are many more ways of carrying out the cleaning process.  
I hope this will be helpful for the readers in some way.  
Thanks for reading :)